# Project: Kinematics Pick & Place

## Steps to complete the project:

1. Set up your ROS Workspace.
2. Download or clone the project repository into the src directory of your ROS Workspace.
3. Experiment with the forward_kinematics environment and get familiar with the robot.
4. Launch in demo mode.
5. Perform Kinematic Analysis for the robot following the project rubric.
6. Fill in the IK_server.py with your Inverse Kinematics code.

Now I will explain each point in detail according to the rubric points.

## Rubric Points

### Writeup / README

1. Provide a Writeup / README that includes all the rubric points and how you addressed each one. You can submit your writeup as markdown or pdf.

    * I decided to use Jupyter book as the format of my writeup. 
    * I am using VMWare on a macbook pro, with my virtual machine image updated from V2.0.1 to V2.1.0. I have encountered several error when I was trying run the demo and I was told by the experts that this was due to the old version of my image file. 
    * I have setup my ROS workspace following the instruction from the lesson.
    * I have cloned the project into the correspoding directory. After I have finished the project I created a repository in my personal Github account and pushed all my work, along with this notebook, to that repository.

### Kinematic Analysis

1. Run the forward_kinematics demo and evaluate the kr210.urdf.xacro file to perform kinematic analysis of Kuka KR210 robot and derive its DH parameters.

    Here are the steps I took to assign the reference frames for the Kuka arm, shown in a series of hand drawings.

![000](imgs/000.jpg)

![001](imgs/001.jpg)

![002](imgs/002.jpg)

![003](imgs/003.jpg)

![004](imgs/004.jpg)

![005](imgs/005.jpg)

![006](imgs/006.jpg)

![007](imgs/007.jpg)

![008](imgs/008.jpg)

To derive the DH parameters, we need to refer to the URDF file. The relevant section is shown below. As marked by the drawings above, the a's and d's are indicated by the origin offsets of the joints. 

Extract the offsets and mark them on the last drawing we can get the following:

![009](imgs/009.jpg)

Fill them into the DH table:

![010](imgs/010.jpg)

Formalize it we can get the following table:

Links | alpha(i-1) | a(i-1) | d(i) | theta(i)
--- | --- | --- | --- | ---
0->1 | 0 | 0 | 0.75 | q1
1->2 | -pi/2 | 0.35 | 0 | -pi/2 + q2
2->3 | 0 | 1.25 | 0 | 0
3->4 | -pi/2 | -0.054 | 1.50 | 0
4->5 | pi/2 | 0 | 0 | 0
5->6 | -pi/2 | 0 | 0 | 0
6->EE | 0 | 0 | 0.303 | 0

2. Using the DH parameter table you derived earlier, create individual transformation matrices about each joint. In addition, also generate a generalized homogeneous transform between base_link and gripper_link using only end-effector(gripper) pose.

For the purpos of clearier code, I have defined a helper function to generate homogeneous transform from the DH parameters.

In [2]:
# a helper function to generate homogeneous transforms from DH parameters
def GetHTFromDH(q, d, a, alpha):
    # q, d, a, alpha are DH parameters
    T = Matrix([
                [           cos(q),           -sin(q),           0,             a],
                [sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
                [sin(q)*sin(alpha), cos(q)*sin(alpha),  cos(alpha),  cos(alpha)*d],
                [                0,                 0,           0,             1]
               ])
    return T

Then it's just a matter of list all the transforms and then sub in the DH parameters.

In [ ]:
# Define Modified DH Transformation matrix
T0_1 = GetHTFromDH(q1, d1, a0, alpha0)
T0_1 = T0_1.subs(s)

T1_2 = GetHTFromDH(q2, d2, a1, alpha1)
T1_2 = T1_2.subs(s)

T2_3 = GetHTFromDH(q3, d3, a2, alpha2)
T2_3 = T2_3.subs(s)

T3_4 = GetHTFromDH(q4, d4, a3, alpha3)
T3_4 = T3_4.subs(s)

T4_5 = GetHTFromDH(q5, d5, a4, alpha4)
T4_5 = T4_5.subs(s)

T5_6 = GetHTFromDH(q6, d6, a5, alpha5)
T5_6 = T5_6.subs(s)

T6_G = GetHTFromDH(q7, d7, a6, alpha6)
T6_G = T6_G.subs(s)

For the generalized homogeneous transform, we can get from the position and orientation of the end effector.

In [ ]:
# Extract end-effector position and orientation from request
# px,py,pz = end-effector position
# roll, pitch, yaw = end-effector orientation
px = req.poses[x].position.x
py = req.poses[x].position.y
pz = req.poses[x].position.z

(roll, pitch, yaw) = tf.transformations.euler_from_quaternion(
    [req.poses[x].orientation.x, req.poses[x].orientation.y,
        req.poses[x].orientation.z, req.poses[x].orientation.w])

# Construct rot for ee symbolically
r, p, y = symbols('r p y')

R_ee_x = Matrix([
              [1,         0,          0],
              [0, cos(r), -sin(r)],
              [0, sin(r),  cos(r)]
              ])
R_ee_y = Matrix([
              [cos(p),  0, sin(p)],
              [0,           1,          0],
              [-sin(p), 0, cos(p)]
              ])
R_ee_z = Matrix([
             [cos(y), -sin(y), 0],
             [sin(y),  cos(y), 0],
             [       0,         0, 1]
             ])
R_ee = R_ee_z * R_ee_y * R_ee_x

We need also consider the discrepancy between DH parameters and Gazebo.

In [ ]:
# Compensate for rotation discrepancy between DH parameters and Gazebo
# get transform for z-axis
R_z = Matrix([
            [cos(pi), -sin(pi), 0],
            [sin(pi),  cos(pi), 0],
            [      0,        0, 1]
            ])
# get transform for y-axis
R_y = Matrix([
            [ cos(-pi/2), 0, sin(-pi/2)],
            [          0, 1,          0],
            [-sin(-pi/2), 0, cos(-pi/2)]
            ])
# get the compensate transform
#R_comp = simplify(R_z * R_y)
R_comp = R_z * R_y

    Combine them we will get the rotation matrix for the end effector.

In [ ]:
R_ee = R_ee * R_comp

What worth noting is that in the script, the construction of the matrix was moved out of the loop where the position and rotation data come in, as a way to optimize the computation time. 

3. Decouple Inverse Kinematics problem into Inverse Position Kinematics and inverse Orientation Kinematics; doing so derive the equations to calculate all individual joint angles.

As we were taught in the lesson, the first angle can be computed directly from the position data of the wrist center:

In [ ]:
theta1 = atan2(WC[1], WC[0])

And the position of the wrist center can be derived as shown in the lesson. What's more tricky is theta 2 and theta 3. The picture shown in the lesson was very misleading by align the axis of both theta 2 and theta 3, and I was confused since in this case, theta 2 and theta 3 would always be the same degree. 

![011](imgs/011.png)

I spent a considerable time trying to make it work under the false assumption that these two angles were equal, until my logic convinced me that this is impossible and had me consulted one of the experts. 

It turns out, for theta 2, things are much easier if you turn your view angel from top down, not side ways in the X-Z panel, but rather in the X-Y panel. That means the following drawing, instead of the above one, should be the correct one to start with.

![012](imgs/012.jpg)

In the drawing, the key point is to realize that the side B can be computed by the position data of wrist center and the DH parameters. Once this clicked, everything else follows naturally, as well as the computation for theta 3.

The computation for theta 4, theta 5 and theta 6 follows the instruction from lesson 2-8, assuming that you perform a extrinsic rotation in the sequence of x-y-z.   

In [ ]:
# Quiz code from lesson 2-8
# define r11, r12, r13
r11 = R_XYZ.row(0)[0]
r12 = R_XYZ.row(0)[1]
r13 = R_XYZ.row(0)[2]

# define r21, r22, r23
r21 = R_XYZ.row(1)[0]
r22 = R_XYZ.row(1)[1]
r23 = R_XYZ.row(1)[2]

# define r31, r32, r33
r31 = R_XYZ.row(2)[0]
r32 = R_XYZ.row(2)[1]
r33 = R_XYZ.row(2)[2]

# Conversion Factors
rtd = 180./np.pi # radians to degrees
dtr = np.pi/180. # degrees to radians

######## TO DO ##########
# Calculate the Euler angles that produces a rotation equivalent to R (above)
# NOTE: Be sure your answer has units of DEGREES!
alpha = atan2(r21, r11) * rtd # rotation about Z-axis
beta  = atan2(-r31, sqrt(r11*r11 + r21*r21)) * rtd # rotation about Y-axis
gamma = atan2(r32, r33) * rtd # rotation about X-axis


During test, I found that the angles derived from this code is not always optimal, so I changed the way to extract the angles.

### Project Implementation

1. Fill in the IK_server.py file with properly commented python code for calculating Inverse Kinematics based on previously performed Kinematic Analysis. Your code must guide the robot to successfully complete 8/10 pick and place cycles. Briefly discuss the code you implemented and your results.

I have already explained the most important parts of the code above. Here I want to address some other aspects of this project.

#### Problems

   1. Failed to start the demo.
   I encountered this problem after I have played the demo successfully the first time. As shown in the following screenshots, the system always complains about some invalid launch file or failing to start some local process. 
   It turns out, that I was using an outdated version of the virtual machine. This problem was gone after I re-downloaded the image and started fresh in V2.1.0.
   
   2. Failed to call service calculate_ik.
   I encountered this problem after I have completed my IK_server.py code and was ready to test it. The arm will always move randomly and then I noticed this error in the terminal. 
   The reason, as simple as it was, was that I didn't start the IK server. Everything starts to work after the service was started.
   
   3. Very slow computation
   For the first several tests, the calculation inverse kinematics takes forever. A closer examination to the code reveals that, it spent too much time constructing the transform matrix and simplify them. I tried my best to move the constructions outside of the loop, only sub values in inside it. This has increased the speed greatly, but not enough. For a normal pick and place, the inverse kinematics step still take about 10~30 seconds to complete.  
   
   4. The holy Next and the forbidden Continue
   When I was testing the arm, I was using the Continue button at first. I never realized that this would actually create a problem. The arm will go to the planned destination and reach the object, close the gripper, but never actually grab it. Then it will act as if it has reached the dropbox and release the object right at the same spot. 
   I spent way too much time trying to figure out what caused this. After making sure that my code is logically sound, I loaded the demo, and found out in shock that the test demo does exactly the same thing. It was until then that I realized there might be something else besides my code that's causing this problem. 
   During a session with the expert, I was told that I should never use the Continue button, only the Next. After I switch to use Next button exclusively, this problem was gone immediately. 

![013](imgs/013.png)

![014](imgs/014.png)

![015](imgs/015.png)

![016](imgs/016.png)

#### Improvements

   1. Speed up the computation
   
   As I mentioned above, I have tried my best to boost the computation speed, including the code optimization and also allocating more resources for the virtual machine. But more advanced algorithms and data structures, like class and pre constructed matrix, could have better effects on the calculation.
   
   2. Smoothier movement
   Currently, the arm moves in a rough point-to-point process. A better algorithm should produce a much smoothier curve for the movement.
   
   3. Real isolation of the position and rotation for the end effector
   In current setup of the arm, all the angles for the 6 joints are output to the simulator all the time, which make the last 3 joints moves, in a seemingly random way, all the time. An optimized sequence should be: the first 3 joints will move first, moving the wrist center to its final destination; then the last 3 joints will rotate so that the end effector is at its rightful place. 